In [ ]:
#| default_exp base

In [ ]:
#| export
import requests
import urllib3
import ssl

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from importlib import reload

# Base

> This is the base notebook for the istatapi package. It contains the code to make the API calls.

In [ ]:
#| export
class ISTAT:
    """Base class that provides useful functions to communicate with ISTAT API"""

    def __init__(self):
        self.base_url = "http://sdmx.istat.it/SDMXWS/rest"
        self.agencyID = "IT1"

    def _request(self, path, **kwargs):
        """Make a request to ISTAT API given a 'path'"""
        if "headers" in kwargs.keys():
            headers = kwargs["headers"]
        else:
            headers = {}
        url = "/".join([self.base_url, path])
        headers["User-Agent"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"
        # response = requests.get(url, headers=headers, verify = False)
        session = get_custom_ssl_session()
        response = session.get(url, headers=headers)
        return response

class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    '''Transport adapter" that allows us to use custom ssl_context.'''

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)
        
def get_custom_ssl_session():
    '''Get a session with a custom ssl context'''
    session = requests.session()        
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [ ]:
show_doc(ISTAT._request)

---

[source](https://github.com/Attol8/istatapi/blob/master/istatapi/base.py#L17){target="_blank" style="float:right; font-size:smaller"}

### ISTAT._request

>      ISTAT._request (path, **kwargs)

Make a request to ISTAT API given a 'path'

The 'ISTAT' class already implements the 'base_url', therefore 'path' only needs to include the other parameters of the request. Here is an example of a succesful request:

In [ ]:
istat = ISTAT()
path = "dataflow/IT1"
test_response = istat._request(path=path)
test_response

<Response [200]>

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()